In [123]:
#works with networkx 2.4 - upgrade to 2.4
#!pip install --upgrade networkx 

import pandas as pd
import networkx as nx
from networkx.algorithms.traversal.depth_first_search import dfs_tree

print(nx.__version__)

2.4


In [89]:
f = open("Predictions_3_files/DCST/poetry_final.conll").read().split("\n\n")
headers = f[0].split("\t")

headDict = dict()
for item in headers:
    headDict[item] = ""
headers

['word_id',
 'word',
 'postag',
 'lemma',
 'gold_head',
 'gold_label',
 'pred_head',
 'pred_label']

In [96]:
sent = f[1]
lines = sent.split("\n")
G=nx.DiGraph()
attDict = dict()
relDict = dict()
for item in [str(i) for i in range(len(lines)+1)]:
    attDict[item] = headDict
G.add_nodes_from([str(i) for i in range(len(lines)+1)])
for line in lines:
    cols = line.split("\t")
    for i,item in enumerate(headers):
        attDict[cols[0]][item] = cols[i]
    G.add_edge(cols[4],cols[0]) # change cols[4] to cols[6] for getting predicted tree
    relDict[(cols[4],cols[0])] = {"head_label":cols[5]} #change cols[5] to cols[7] for getting predicted tree. Also 4 to 6
nx.set_node_attributes(G, attDict)
nx.set_edge_attributes(G, relDict)
list(G.edges(data=True))

[('0', '1', {'head_label': 'root'}),
 ('1', '2', {'head_label': 'sambanxah'}),
 ('1', '3', {'head_label': 'karma'}),
 ('1', '6', {'head_label': 'karwa'}),
 ('1', '7', {'head_label': 'axikaranam'}),
 ('3', '8', {'head_label': 'rartisambanxah'}),
 ('7', '4', {'head_label': 'viseranam'}),
 ('7', '5', {'head_label': 'viseranam'})]

In [103]:
data = nx.tree_data(G, root="0")
tree = nx.tree_graph(data)

In [109]:
tree["1"]

AtlasView({'2': {}, '3': {}, '6': {}, '7': {}})

In [117]:
list(G.predecessors("2")), list(G.successors("1"))

(['1'], ['2', '3', '6', '7'])

In [244]:
from networkx.algorithms.traversal.depth_first_search import dfs_tree

def optimalDL(depTree, rootNode = "0"):
    assert nx.is_tree(G) 

    currNodeList = [rootNode]
    levelOrder = dict()
    while len(currNodeList) >0:
        currNode = currNodeList.pop(0)
        
        succNodes = list(depTree.successors(currNode))
        
        if len(succNodes) == 0:
            continue
        
        succChildSize = dict()
        for item in succNodes:
            succChildSize[item] = len(dfs_tree(depTree,item).nodes())
            currNodeList.append(item)
        
        succChildSize = {k: v for k, v in sorted(succChildSize.items(), key=lambda item: item[1])}
 
        
        left = 1
        _Order = [currNode]
        for item in succChildSize:
            
            if left == 1:
                _Order.insert(0,item)
                left = 0
            else:
                _Order.append(item)
                left = 1
        levelOrder[currNode] = _Order
    
    


    explored = set()
    explored.add("0")
    toExplore = list(set(levelOrder["0"]) - explored)
    finalOrder = levelOrder["0"]

    

    while len(toExplore) >0:
        a = toExplore.pop()
        explored.add(a)
        for i,item in enumerate(finalOrder):
            if item == a:
                finalOrder[i] = levelOrder[a]
        _ = list()
        for item in finalOrder:
            if type(item) == list:
                for stuff in item:
                    _.append(stuff)
            else:
                _.append(item)
        
        finalOrder = _.copy()
        
        for item in levelOrder[a]:
            if item in levelOrder.keys():
                if item not in explored:
                    toExplore.append(item)
        
    assert  len(finalOrder) == len(set(finalOrder))
    return finalOrder

In [246]:
def depLen(inpGraph, order):
    """
    Given a dependnecy tree and the linear order in which the words in the tsentence are arranged,
    we calcualte the dependency length of the sentence
    
    inpGraph = tree as nx diGraph
    order = list of word ids, based on their order (can be optimal, prose or poetry order)
    """
    assert nx.is_tree(inpGraph)
    print(order)
    toExplore = ["0"]
    dl = 0
    while len(toExplore) > 0:
        a = toExplore.pop()
        sourIndex = order.index(a)
        succNodes  = list(inpGraph.successors(a))
        for item in succNodes:
            tarIndex = order.index(item)
            pairDiff = abs(sourIndex-tarIndex)
            dl += pairDiff
            if len(list(inpGraph.successors(item))) > 0:
                toExplore.append(item)
            print(dl, "from: " , a,"to:", item, "palced apart:",pairDiff, toExplore, succNodes)
    return dl

In [245]:
optdl = optimalDL(G)
depLen(G,optdl)

['8', '3', '2', '1', '6', '4', '7', '5', '0']
5 from:  0 to: 1 palced apart: 5 ['1'] ['1']
6 from:  1 to: 2 palced apart: 1 [] ['2', '3', '6', '7']
8 from:  1 to: 3 palced apart: 2 ['3'] ['2', '3', '6', '7']
9 from:  1 to: 6 palced apart: 1 ['3'] ['2', '3', '6', '7']
12 from:  1 to: 7 palced apart: 3 ['3', '7'] ['2', '3', '6', '7']
13 from:  7 to: 4 palced apart: 1 ['3'] ['4', '5']
14 from:  7 to: 5 palced apart: 1 ['3'] ['4', '5']
15 from:  3 to: 8 palced apart: 1 [] ['8']


15

In [187]:
list(dfs_tree(G,"0"))

['0', '1', '2', '3', '8', '6', '7', '4', '5']

In [143]:

subtree_at_2 = dfs_tree(G, "1")
len(subtree_at_2.nodes())

8

In [156]:
a= [5]
a.insert(0,4)
a.append(3)
a.pop(0),a

(4, [5, 3])